# ✈️ Travel Planner Assistant

This notebook sets up the virtual environment, installs dependencies, and runs the multi-agent travel planner powered by AutoGen 0.4+.

**Architecture:** SelectorGroupChat with 5 specialist agents + human-in-the-loop, backed by Groq Cloud (Llama 3.3 70B) and persistent memory.

## 1. Install Dependencies

Install all required packages from `requirements.txt` into the current kernel environment.

In [ ]:
import sys
%pip install -r requirements.txt --quiet

## 2. Verify Installation

Quick check that core packages imported successfully.

In [ ]:
import autogen_agentchat
import autogen_ext
import dotenv

print(f"autogen-agentchat: {autogen_agentchat.__version__}")
print(f"autogen-ext:       {autogen_ext.__version__}")
print("✅ All dependencies installed successfully.")

## 3. Authentication

**Groq (default):** No Azure login needed — the Groq API key in `.env` is used directly.
**Azure OpenAI:** Uncomment and run the `az login` cell below if you set `LLM_PROVIDER=azure`.

In [ ]:
# Only needed if LLM_PROVIDER=azure  (skip for Groq)
# !az login --output table

## 4. Load Environment & Memory

Load the `.env` file (Azure OpenAI config) and any previously saved user preferences.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Display configuration (uses DefaultAzureCredential - no API key needed)
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT', '❌ NOT SET')
model = os.getenv('AZURE_OPENAI_MODEL_NAME') or os.getenv('AZURE_OPENAI_DEPLOYMENT') or '❌ NOT SET'
api_version = os.getenv('AZURE_OPENAI_API_VERSION', '❌ NOT SET')

print(f"Endpoint:    {endpoint}")
print(f"Model:       {model}")
print(f"API Version: {api_version}")
print(f"\n🔐 Auth: DefaultAzureCredential")

from config.memory import load_memory

memory = load_memory()
if memory:
    print(f"\n📝 Remembered preferences: {memory}")
else:
    print("\n📝 No saved preferences yet (first run).")

## 5. Build the Agent Team

Constructs the SelectorGroupChat with all 5 specialist agents (planner, flight, hotel, weather, itinerary) plus the UserProxyAgent for human-in-the-loop.

In [ ]:
from agents.team import build_team

team = build_team()
print("✅ Agent team built successfully.")

## 6. Run a Travel Query

Edit the `query` variable below with your travel request, then run the cell.

**Note:** If the planner asks a clarifying question, you will be prompted for input in the cell output.

In [ ]:
import json
import re
from autogen_agentchat.ui import Console
from config.memory import save_memory


def _extract_preferences(messages: list) -> dict:
    """Scan agent messages for SAVE_PREFERENCE markers and extract them."""
    preferences = {}
    pattern = re.compile(r"SAVE_PREFERENCE:\s*(\{.*?\})", re.DOTALL)
    for msg in messages:
        content = getattr(msg, "content", "") or ""
        if not isinstance(content, str):
            continue
        for match in pattern.finditer(content):
            try:
                prefs = json.loads(match.group(1))
                preferences.update(prefs)
            except json.JSONDecodeError:
                pass
    return preferences


# ──────────────────────────────────────────────
# ✏️  EDIT YOUR TRAVEL QUERY HERE
# ──────────────────────────────────────────────
query = "Plan a 5-day trip to Tokyo in April for a couple who loves food and culture"
# ──────────────────────────────────────────────

print(f"🗒️  Query: {query}\n")

result = await Console(team.run_stream(task=query))

# Extract and persist any preferences the planner flagged
new_prefs = _extract_preferences(result.messages)
if new_prefs:
    save_memory(new_prefs)
    print(f"\n📝 Saved preferences: {new_prefs}")

## 7. Ask a Follow-up Question

Reset the team and run another query. Edit `follow_up` below and run the cell.

In [ ]:
# Reset team for a fresh conversation (memory persists across runs)
await team.reset()

# ──────────────────────────────────────────────
# ✏️  EDIT YOUR FOLLOW-UP QUERY HERE
# ──────────────────────────────────────────────
follow_up = "Now plan the same trip but for Paris instead"
# ──────────────────────────────────────────────

print(f"🗒️  Query: {follow_up}\n")

result = await Console(team.run_stream(task=follow_up))

new_prefs = _extract_preferences(result.messages)
if new_prefs:
    save_memory(new_prefs)
    print(f"\n📝 Saved preferences: {new_prefs}")

## 8. Inspect Saved Memory

View the preferences that have been persisted to `memory/preferences.json`.

In [ ]:
from config.memory import load_memory, format_memory_context
import json

memory = load_memory()
if memory:
    print(json.dumps(memory, indent=2))
    print("\n--- Formatted context injected into planner ---")
    print(format_memory_context(memory))
else:
    print("No preferences saved yet.")